In [1]:
import pandas as pd
import numpy as np

# Code requirements

- Create deck
- Shuffle deck
- Deal a card
- Save dealer and player hands
- Calculate points
    - Deal with Aces
- Simulate a game
    - Deal with whether the player should hit
    - Deal with whether the dealer needs to hit
- Simulate trials
    - To find expected wins
- Save our data into a dataframe

# Deck

In [2]:
class Deck():
    values = ["A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K"]
    def __init__(self, num_decks=1, values=values):
        # Multiply values by number of suits
        # Then by number of decks
        self.num_decks = num_decks
        self.deck = values * 4 * self.num_decks
        
    def __str__(self):
        return "{} decks, {} cards left".format(self.num_decks, len(self.deck))
    
    def shuffle(self):
        np.random.shuffle(self.deck)
    
    def deal(self, hand=None):
        if hand == None:
            return self.deck.pop(0)
        else:
            hand.append(self.deck.pop(0))

In [3]:
test = Deck()
print (test)
print (test.deck)
print (test.deal())
print (test.deck)
test.shuffle()
print (test.deck)

1 decks, 52 cards left
['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']
A
[2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']
[10, 'Q', 2, 10, 'K', 8, 7, 8, 7, 9, 5, 4, 3, 'A', 6, 'J', 3, 'Q', 8, 'A', 2, 8, 7, 9, 7, 'J', 4, 3, 'J', 2, 6, 4, 5, 'Q', 6, 'K', 'A', 3, 2, 'K', 10, 4, 6, 9, 10, 5, 'J', 'Q', 5, 'K', 9]


# Calculate points

In [4]:
def calculate_points(hand):
    points = 0
    num_ace = 0
    
    # Deal with aces
    while "A" in hand:
        hand.remove("A")
        num_ace += 1
    
    # Deal with the rest of the hand
    for i in hand:
        try:
            # If it's a number, add it to points
            points += i
        except:
            # If it's not a number, it's a 10
            points += 10
            
    # deal with the aces
    for i in range(num_ace):
        if points + 11 <= 21:
            points += 11
        else:
            points += 1
    return points

In [5]:
calculate_points([5, 5, "A"])

21

# Simulate game

In [6]:
data_dictionary = pd.read_csv("data_dictionary.csv")
data_dictionary

,Feature,Type,Description
0,num_decks,Int,Number of decks used
1,dealer_open,Int,The card we can see
2,dealer_initial,Int,The dealer's starting points
3,dealer_hit,Binary,"1 - dealer hit, 0 - dealer did not hit"
4,dealer_num_hits,Int,Number of times the dealer hit
5,dealer_final,Int,The dealer's final points
6,dealer_busts,Binary,"1 - dealer busts, 0 - dealer did not bust"
7,player_card_one,Int,Player's first card
8,player_card_two,Int,Player's second card
9,player_initial,Int,The player's starting points


In [7]:
def sim_game(num_decks=1, strategy=0):
    game_deck = Deck(num_decks=num_decks)
    game_deck.shuffle()

    dealer_hand = []
    player_hand = []

    # deal 2 cards each
    for i in range(2):
        game_deck.deal(player_hand)
        game_deck.deal(dealer_hand)

    # get their initial points
    # copy so that our hand is not changed
    player_initial = calculate_points(player_hand.copy())
    dealer_initial = calculate_points(dealer_hand.copy())

    # get dealer open card
    dealer_open = dealer_hand[0]

    # get cards from player
    player_card_one = player_hand[0]
    player_card_two = player_hand[1]

    # Change them to numerics
    if dealer_open in ["J", "Q", "K"]:
        dealer_open = 10
    elif dealer_open == "A":
        dealer_open = 1

    if player_card_one in ["J", "Q", "K"]:
        player_card_one = 10
    elif player_card_one == "A":
        player_card_one = 1

    if player_card_two in ["J", "Q", "K"]:
        player_card_two = 10
    elif player_card_two == "A":
        player_card_two = 1

    dealer_hit = 0
    dealer_num_hits = 0
    player_hit = 0
    player_num_hits = 0
    player_busts = 0
    dealer_busts = 0
    dealer_final = calculate_points(dealer_hand.copy())
    player_final = calculate_points(player_hand.copy())


    # if anyone got a blackjack, the game should end
    if player_initial != 21 and dealer_initial != 21:
        # if neither of them got a blackjack game continues
        # the player goes first

        # If player <= 11, hit
        while calculate_points(player_hand.copy()) <= 11:
            player_hit = 1
            player_num_hits += 1
            game_deck.deal(player_hand)

        # If strategy is random, randomize hit for 18 and below
        if strategy == 0:
            while calculate_points(player_hand.copy()) <= 18:
                if np.random.random() < 0.5:
                    player_hit = 1
                    player_num_hits += 1
                    game_deck.deal(player_hand)
        # If strategy is recommended, stand on 17 and above
        else:
            if dealer_open <= 6:
                while calculate_points(player_hand.copy()) <= 16:
                    player_hit = 1
                    player_num_hits += 1
                    game_deck.deal(player_hand)

        # update player's final and busts
        player_final = calculate_points(player_hand.copy())
        player_busts = player_final > 21
        # dealer's turn

        # If player didn't bust
        if not player_busts:
            # If dealer < 17, hit
            while calculate_points(dealer_hand.copy()) < 17:
                dealer_hit = 1
                dealer_num_hits += 1
                game_deck.deal(dealer_hand)

            # If dealer has a soft 17, hit
            if calculate_points(dealer_hand.copy()) == 17:
                if "A" in dealer_hand:
                    dealer_copy = dealer_hand.copy()
                    dealer_copy.remove("A")
                    if calculate_points(dealer_copy.copy()) == 6:
                        dealer_hit = 1
                        dealer_num_hits += 1
                        game_deck.deal(dealer_hand)

        # update dealer's final and busts
        dealer_final = calculate_points(dealer_hand.copy())
        dealer_busts = dealer_final > 21

    player_loses = 0
    draw = 0
    player_wins = 0

    # Check who wins
    if player_initial == 21 and dealer_initial != 21:
        player_wins = 1
    elif dealer_initial == 21 and player_initial != 21:
        player_loses = 1
    elif player_busts:
        player_loses = 1
    elif dealer_busts:
        player_wins = 1
    elif player_final > dealer_final:
        player_wins = 1
    elif player_final < dealer_final:
        player_loses = 1
    elif player_final == dealer_final:
        draw = 1

    # Change hands to strings
    dealer_hand_str = ",".join([str(i) for i in dealer_hand])
    player_hand_str = ",".join([str(i) for i in player_hand])
    
    return np.array([num_decks, dealer_open, dealer_initial, 
                     dealer_hit, dealer_num_hits, dealer_final, 
                     int(dealer_busts), player_card_one, 
                     player_card_two, player_initial, player_hit, 
                     player_num_hits, player_final, int(player_busts), 
                     player_loses, draw, player_wins, strategy, 
                     dealer_hand_str, player_hand_str])

In [8]:
pd.Series(sim_game(), index=data_dictionary.Feature.values)

num_decks                 1
dealer_open               3
dealer_initial            7
dealer_hit                0
dealer_num_hits           0
dealer_final              7
dealer_busts              0
player_card_one           1
player_card_two           3
player_initial           14
player_hit                1
player_num_hits           2
player_final             22
player_busts              1
player_loses              1
draw                      0
player_wins               0
strategy                  0
dealer_hand             3,4
player_hand        A,3,10,8
dtype: object

# Generate a dataframe

In [9]:
data_dictionary.Feature.values

array(['num_decks', 'dealer_open', 'dealer_initial', 'dealer_hit',
       'dealer_num_hits', 'dealer_final', 'dealer_busts',
       'player_card_one', 'player_card_two', 'player_initial',
       'player_hit', 'player_num_hits', 'player_final', 'player_busts',
       'player_loses', 'draw', 'player_wins', 'strategy', 'dealer_hand',
       'player_hand'], dtype=object)

In [10]:
def gen_data(num_decks=1, df_size=5000, strategy=0):
    return np.array([sim_game(num_decks=num_decks, strategy=strategy) for _ in range(df_size)])

In [11]:
def gen_df(data):
    tmp = pd.DataFrame(data, columns=data_dictionary.Feature.values)
    tmp[tmp.columns.values[:-2]] = tmp[tmp.columns.values[:-2]].astype(int)
    return tmp

In [54]:
ran = gen_df(gen_data(num_decks=4))

In [55]:
rec = gen_df(gen_data(num_decks=4, strategy=1))

In [56]:
df = pd.concat([ran,rec])

In [57]:
# Combine old files if they are there
try:
    old_df = pd.read_csv("blackjack.csv")
    df = pd.concat([old_df, df])
except:
    pass

In [58]:
# save to file
df.to_csv("blackjack.csv", index=False)

# Testing our code

In [42]:
df = pd.read_csv("blackjack.csv")

In [43]:
df.shape

(10000, 20)

In [19]:
# When player_loses = 1, draw = 0, player_wins = 0
print (df.draw[df.player_loses == 1].value_counts())
print (df.player_wins[df.player_loses == 1].value_counts())

0    43636
Name: draw, dtype: int64
0    43636
Name: player_wins, dtype: int64


In [20]:
# When player_loses = 0, draw = 1, player_wins = 0
print (df.player_loses[df.draw == 1].value_counts())
print (df.player_wins[df.draw == 1].value_counts())

0    5759
Name: player_loses, dtype: int64
0    5759
Name: player_wins, dtype: int64


In [21]:
# When player_loses = 0, draw = 0, player_wins = 1
print (df.player_loses[df.player_wins == 1].value_counts())
print (df.draw[df.player_wins == 1].value_counts())

0    30605
Name: player_loses, dtype: int64
0    30605
Name: draw, dtype: int64


In [22]:
# When dealer_busts = 1, player_loses = 0, draw = 0, player_wins = 1
print (df.player_loses[df.dealer_busts == 1].value_counts())
print (df.draw[df.dealer_busts == 1].value_counts())
print (df.player_wins[df.dealer_busts == 1].value_counts())

0    14260
Name: player_loses, dtype: int64
0    14260
Name: draw, dtype: int64
1    14260
Name: player_wins, dtype: int64


In [23]:
# When player_busts = 1, player_loses = 1, draw = 0, player_wins = 0, dealer_hit = 0, dealer_busts = 0
print (df.player_loses[df.player_busts == 1].value_counts())
print (df.draw[df.player_busts == 1].value_counts())
print (df.player_wins[df.player_busts == 1].value_counts())
print (df.dealer_hit[df.player_busts == 1].value_counts())
print (df.dealer_busts[df.player_busts == 1].value_counts())

1    23676
Name: player_loses, dtype: int64
0    23676
Name: draw, dtype: int64
0    23676
Name: player_wins, dtype: int64
0    23676
Name: dealer_hit, dtype: int64
0    23676
Name: dealer_busts, dtype: int64


In [24]:
# When dealer_busts = 0 and player_busts = 0, if player_final > dealer final, player_loses = 0, draw = 0, player_wins = 1
sub = df[(df.player_busts == 0) & (df.dealer_busts == 0)]
print (sub.player_loses[sub.player_final > sub.dealer_final].value_counts())
print (sub.draw[sub.player_final > sub.dealer_final].value_counts())
print (sub.player_wins[sub.player_final > sub.dealer_final].value_counts())

0    16345
Name: player_loses, dtype: int64
0    16345
Name: draw, dtype: int64
1    16345
Name: player_wins, dtype: int64


In [25]:
# When dealer_busts = 0 and player_busts = 0, if player_final = dealer final, player_loses = 0, draw = 1, player_wins = 0
print (sub.player_loses[sub.player_final == sub.dealer_final].value_counts())
print (sub.draw[sub.player_final == sub.dealer_final].value_counts())
print (sub.player_wins[sub.player_final == sub.dealer_final].value_counts())

0    5759
Name: player_loses, dtype: int64
1    5759
Name: draw, dtype: int64
0    5759
Name: player_wins, dtype: int64


In [26]:
# When dealer_busts = 0 and player_busts = 0, if player_final < dealer final, player_loses = 1, draw = 0, player_wins = 0
print (sub.player_loses[sub.player_final < sub.dealer_final].value_counts())
print (sub.draw[sub.player_final < sub.dealer_final].value_counts())
print (sub.player_wins[sub.player_final < sub.dealer_final].value_counts())

1    19960
Name: player_loses, dtype: int64
0    19960
Name: draw, dtype: int64
0    19960
Name: player_wins, dtype: int64


In [27]:
# When player_final <= 21, player_busts == 0
print (df.player_busts[df.player_final <= 21].value_counts())

0    56324
Name: player_busts, dtype: int64


In [28]:
# When dealer_final <= 21, dealer_busts == 0
print (df.dealer_busts[df.dealer_final <= 21].value_counts())

0    65740
Name: dealer_busts, dtype: int64


In [29]:
# When player_hit == 1, player_num_hits >= 1
print (df.player_num_hits[df.player_hit == 1].value_counts())

1    28574
2    13320
3     3290
4      483
5       54
6        4
Name: player_num_hits, dtype: int64


In [30]:
# When dealer_hit == 1, dealer_num_hits >= 1
print (df.dealer_num_hits[df.dealer_hit == 1].value_counts())

1    22435
2     8415
3     1580
4      165
5        9
Name: dealer_num_hits, dtype: int64


In [31]:
df.dealer_hit.value_counts()

0    47396
1    32604
Name: dealer_hit, dtype: int64

In [32]:
df.dealer_num_hits.value_counts()

0    47396
1    22435
2     8415
3     1580
4      165
5        9
Name: dealer_num_hits, dtype: int64

In [33]:
df.player_hit.value_counts()

1    45725
0    34275
Name: player_hit, dtype: int64

In [34]:
df.player_num_hits.value_counts()

0    34275
1    28574
2    13320
3     3290
4      483
5       54
6        4
Name: player_num_hits, dtype: int64

In [35]:
df[["dealer_final", "player_final"]][df.strategy == 0].describe()

,dealer_final,player_final
count,40000.000000,40000.000000
mean,17.215700,21.826775
std,4.517459,3.238932
min,4.000000,4.000000
25%,14.000000,20.000000
50%,18.000000,21.000000
75%,20.000000,24.000000
max,26.000000,28.000000


In [36]:
df[["dealer_final", "player_final"]][df.strategy == 1].describe()

,dealer_final,player_final
count,40000.000000,40000.000000
mean,18.969350,17.953550
std,4.026148,3.520391
min,4.000000,4.000000
25%,17.000000,15.000000
50%,19.000000,18.000000
75%,21.000000,20.000000
max,26.000000,26.000000
